In [1]:
import os
import random
import torch
import numpy as np
from nnunet.inference.predict import load_model_and_checkpoint_files
from sklearn.metrics import f1_score

os.system('jupyter nbconvert --to python Pytorch_model.ipynb')
from Pytorch_model import Encodeur_nnUnet, Encodeur_Net

os.system('jupyter nbconvert --to python Pytorch_train_encodeur.ipynb')
from Pytorch_train_encodeur import train, evaluation

seed = 0
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

/home/nguyen-k/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



[NbConvertApp] Converting notebook Pytorch_model.ipynb to python
[NbConvertApp] Writing 30086 bytes to Pytorch_model.py
[NbConvertApp] Converting notebook Pytorch_train_encodeur.ipynb to python
[NbConvertApp] Writing 10749 bytes to Pytorch_train_encodeur.py
[NbConvertApp] Converting notebook Pytorch_dataloader.ipynb to python
[NbConvertApp] Writing 31657 bytes to Pytorch_dataloader.py
[NbConvertApp] Converting notebook preprocessing_Pytorch.ipynb to python
[NbConvertApp] Writing 28850 bytes to preprocessing_Pytorch.py
[NbConvertApp] Converting notebook Pytorch_utils.ipynb to python
[NbConvertApp] Writing 98282 bytes to Pytorch_utils.py
[NbConvertApp] Converting notebook roc_Precision_Recall.ipynb to python
[NbConvertApp] Writing 4174 bytes to roc_Precision_Recall.py


In [2]:
#DATA
dim = (32*4, 32*3, 32*6)
spacing = 4

#IMAGE GENERATOR
scale = (0.80, 1.20)
sigma = (0.3, 0.8)
task = '001'
#001 : PET PUV - 002 : PET sans SUV - 003 - PET CT 2 Channels - 004 - PET CT Fusion

#MODELE
learning_rate = 1e-4
score_weight = 3
drop_encode = 0.5
l1_lambda_fc1, l2_lambda_fc1 = 1e-3, 1e-3
weight_decay = 3e-5

batch_size = 5
nb_epoch = 40
num_workers = 10
patience = 3

In [3]:
base = '/media/nguyen-k/nnUNet_trained_models/nnUNet/3d_fullres'
list_task = os.listdir(base)

for t in list_task :
    if task in t :
        folders = os.path.join(base, t, 'nnUNetTrainerV2__nnUNetPlansv2.1')

In [4]:
for ind_list in range(1) : 

    test = 'Encodeur_PET_L'+str(ind_list)
    dir_base = '/home/nguyen-k/Bureau/segCassiopet2/Comparatif/Archives_Encodeur/'+test
    try:
        os.mkdir(dir_base)
    except OSError as error: 
        print(error) 

    path_list = '/home/nguyen-k/Bureau/segCassiopet2/List_Patient_'+str(ind_list)
    list_test = list(np.load(path_list + '/Test/list_test.npy'))
    test_label_classe = np.load(path_list + '/Test/test_label_classe.npy')   
    test_prob = np.zeros((len(list_test), 3))
    np.save(dir_base+'/list_test.npy', list_test)
    np.save(dir_base+'/test_label_classe.npy', test_label_classe)

    for fold in range(1, 6) :
        print('LIST PATIENT', ind_list, ' - FOLD', fold)  

        dir_p = dir_base+'/Fold'+str(fold)
        dir_p_1 = dir_p+'/Fig_seg_val'
        dir_p_2 = dir_p+'/Fig_seg_test'

        try:
            os.mkdir(dir_p)
        except OSError as error: 
            print('') 

        try:
            os.mkdir(dir_p_1)
        except OSError as error: 
            print('') 

        try:
            os.mkdir(dir_p_2)
        except OSError as error: 
            print('') 

        #TRAIN     
        trainer, params_tr = load_model_and_checkpoint_files(folders, folds=None, mixed_precision=None, checkpoint_name="model_best")
        nn_Unet = Encodeur_nnUnet(trainer.network)
        state_dict = trainer.network.state_dict()
        nn_Unet.load_state_dict(state_dict)
        EncodeurNet = Encodeur_Net(nn_Unet, drop_encode=drop_encode).cuda()   

        train(fold, EncodeurNet, nb_epoch, score_weight, l1_lambda_fc1, l2_lambda_fc1, dim, spacing, scale, sigma, 
                    num_workers, drop_encode, batch_size, learning_rate, patience, weight_decay, dir_p, path_list, seed)
        
        #EVALUATION
        print('LIST PATIENT', ind_list, ' - FOLD', fold, ' - VALIDATION')
        path_train_val = os.path.join(path_list, 'Fold'+ str(fold))
        list_val = list(np.load(path_train_val+'/list_val.npy'))
        val_label_classe = np.load(path_train_val+'/val_label_classe.npy')
        evaluation(EncodeurNet, list_val, val_label_classe, scale, sigma, dim, spacing, num_workers, dir_p_1)

        #TEST
        print('LIST PATIENT', ind_list, ' - FOLD', fold, ' - TEST')
        test_prob = evaluation(EncodeurNet, list_test, test_label_classe, scale, sigma, dim, spacing, num_workers, dir_p_2)
        np.save(os.path.join(dir_p, 'test_prob.npy'), test_prob)
        print(' ')

        del EncodeurNet

[Errno 17] File exists: '/home/nguyen-k/Bureau/segCassiopet2/Comparatif/Archives_Encodeur/Encodeur_PET_L0'
LIST PATIENT 0  - FOLD 1



folds is None so we will automatically look for output folders (not using 'all'!)
found the following folds:  ['/media/nguyen-k/nnUNet_trained_models/nnUNet/3d_fullres/Task001/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0']
using the following model files:  ['/media/nguyen-k/nnUNet_trained_models/nnUNet/3d_fullres/Task001/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0/model_best.model']
LIST PATIENT 0  - FOLD 1  - VALIDATION
Classification
Micro F1 score  0.6666666666666666
Macro F1 score  0.4045584045584045
Weighted F1 score  0.6452991452991452
LIST PATIENT 0  - FOLD 1  - TEST


/home/nguyen-k/Bureau/segCassiopet2/roc_Precision_Recall.py:81: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Classification
Micro F1 score  0.5918367346938775
Macro F1 score  0.41933621933621934
Weighted F1 score  0.5918367346938775
 
LIST PATIENT 0  - FOLD 2
folds is None so we will automatically look for output folders (not using 'all'!)
found the following folds:  ['/media/nguyen-k/nnUNet_trained_models/nnUNet/3d_fullres/Task001/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0']


/home/nguyen-k/Bureau/segCassiopet2/roc_Precision_Recall.py:81: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


using the following model files:  ['/media/nguyen-k/nnUNet_trained_models/nnUNet/3d_fullres/Task001/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0/model_best.model']
LIST PATIENT 0  - FOLD 2  - VALIDATION
Classification
Micro F1 score  0.6666666666666666
Macro F1 score  0.36049382716049383
Weighted F1 score  0.6325102880658435
LIST PATIENT 0  - FOLD 2  - TEST


...

/home/nguyen-k/Bureau/segCassiopet2/roc_Precision_Recall.py:81: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


using the following model files:  ['/media/nguyen-k/nnUNet_trained_models/nnUNet/3d_fullres/Task001/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0/model_best.model']
LIST PATIENT 0  - FOLD 5  - VALIDATION
Classification
Micro F1 score  0.6944444444444444
Macro F1 score  0.38011695906432746
Weighted F1 score  0.6345029239766082
LIST PATIENT 0  - FOLD 5  - TEST


/home/nguyen-k/Bureau/segCassiopet2/roc_Precision_Recall.py:81: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Classification
Micro F1 score  0.6530612244897959
Macro F1 score  0.39493670886075954
Weighted F1 score  0.5850684577628519
 


/home/nguyen-k/Bureau/segCassiopet2/roc_Precision_Recall.py:81: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [6]:
os.system('jupyter nbconvert --to python roc_Precision_Recall.ipynb')
from roc_Precision_Recall import *
#from sklearn.metrics import confusion_matrix

for ind_list in range() : 

    print('LIST PATIENT', ind_list)  

    test = 'Encodeur_PET_L'+str(ind_list)
    dir_base = '/home/nguyen-k/Bureau/segCassiopet2/Comparatif/Archives_Encodeur/'+test

    fold = 1
    dir_p = dir_base+'/Fold'+str(fold)
    sum = np.load(os.path.join(dir_p, 'test_prob.npy'))

    path_list = '/home/nguyen-k/Bureau/segCassiopet2/List_Patient_'+str(ind_list)
    list_test = list(np.load(path_list + '/Test/list_test.npy'))
    test_label_classe = np.load(path_list + '/Test/test_label_classe.npy')   
    test_label_classe = np.array(test_label_classe, dtype=np.uint8)

    for fold in range(2, 6) :
        dir_p = dir_base+'/Fold'+str(fold)
        test_prob = np.load(os.path.join(dir_p, 'test_prob.npy'))
        sum += test_prob

    test_prob = sum / 5

    pred = np.zeros(test_label_classe.shape[0])
    for i in range(test_label_classe.shape[0]) : 
        pred[i] = np.argmax(test_prob[i])

    mat_label = np.zeros((test_label_classe.shape[0],3))
    for i in range(test_label_classe.shape[0]) :
        mat_label[i, int(test_label_classe[i])] = 1

    roc_auc, fpr, tpr = compute_ROC_auc(y_label=mat_label,y_predicted=test_prob,n_classes=3)
    plt.clf()
    plot_ROC_curve(fpr,tpr,roc_auc,classe=0,color='blue')
    plot_ROC_curve(fpr,tpr,roc_auc,classe=1,color='red')
    plot_ROC_curve(fpr,tpr,roc_auc,classe=2,color='black')
    plt.savefig(dir_base+'/ROC.png')

    precision, recall,average_precision = compute_precision_recall(y_label=mat_label,y_predicted=test_prob,n_classes=3)
    plot_precision_recall_curve(precision, recall, average_precision,n_classes=3,color=['blue','red','black'])
    plt.savefig(dir_base+'/AUC.png')

    print('Micro F1 score ', f1_score(y_true=test_label_classe, y_pred=pred, average='micro'))
    print('Macro F1 score ', f1_score(y_true=test_label_classe, y_pred=pred, average='macro'))
    print('Weighted F1 score ', f1_score(y_true=test_label_classe, y_pred=pred, average='weighted'))

    '''matrix = confusion_matrix(test_label_classe, pred)
    accracy_per_class = matrix.diagonal()/matrix.sum(axis=1)
    print('Accuracy C1 = ', accracy_per_class[0])
    print('Accuracy C2 = ', accracy_per_class[1])
    print('Accuracy C3 = ', accracy_per_class[2])'''

[NbConvertApp] Converting notebook roc_Precision_Recall.ipynb to python
[NbConvertApp] Writing 4174 bytes to roc_Precision_Recall.py
/home/nguyen-k/Bureau/segCassiopet2/roc_Precision_Recall.py:81: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


LIST PATIENT 0
Micro F1 score  0.6326530612244898
Macro F1 score  0.25833333333333336
Weighted F1 score  0.5535714285714286
